In [ ]:
# Intersection over union (IoU) is a common evaluation metric for object detection tasks.
#  1. NMS
#  2. mAP

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def computeIOU(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

# main function
def main():
    # define the two bounding boxes
    boxA = (10, 10, 50, 50)
    boxB = (40, 40, 80, 80)

    # compute the intersection over union and display it
    iou = computeIOU(boxA, boxB)
    print("IoU: {:.2f}".format(iou))

    # plot the bounding boxes
    fig, ax = plt.subplots(1)
    ax.add_patch(patches.Rectangle((boxA[0], boxA[1]), boxA[2] - boxA[0], boxA[3] - boxA[1], edgecolor='r', facecolor='none'))
    ax.add_patch(patches.Rectangle((boxB[0], boxB[1]), boxB[2] - boxB[0], boxB[3] - boxB[1], edgecolor='b', facecolor='none'))
    plt.xlim([0, 100])
    plt.ylim([0, 100])
    plt.show()

In [ ]:
# Non-Maximum Suppression

from typing import List, Tuple
import numpy as np

def nms(boxes: np.ndarray, scores: np.ndarray, iou_threshold: float = 0.5) -> List[int]:
    """non maximum suppression """

    # empty boxes candidates checking
    if len(boxes) == 0:
        return []
    # step 1: sort the boxes per socre
    order = np.argsort(scores)[::-1]
    keep = []

    # iterate through the order, calcualte the current box IoU with the other boxes, keep the boxes where iou < iou_threshold (different instance). update the order
    while order.size > 0:
        index = order[0]
        keep.append(index)
        # check if there is only one box remaining
        if order.size == 1:
            break

        ious = batch_iou(boxes[index], boxes[order[1:]])
        indices = np.where(ious <= iou_threshold)[0]
        # update order
        order = order[indices + 1]
    return keep

def batch_iou(box1: np.ndarray, boxes: np.ndarray) -> np.ndarray:
    """
    batch iou calcualtion. assuming boxes with coordinates x1, x2, y1, y2
    """
    # enforce x2 > x1 and y2 > y1
    box1 = np.array([
        min(box1[0], box1[2]),
        min(box1[1], box1[3]),
        max(box1[0], box1[2]),
        max(box1[1], box1[3]),
    ])
    boxes = np.array([
        np.min(boxes[:,0], boxes[:,2]),
        np.min(boxes[:,1], boxes[:,3]),
        np.max(boxes[:,0], boxes[:,2]),
        np.max(boxes[:,1], boxes[:,3]),
    ]).T
    # compute intersection
    x1 = np.maximum(box1[0], boxes[:,0])
    y1 = np.maximum(box1[1], boxes[:,1])
    x2 = np.minimum(box1[2], boxes[:,2])
    y2 = np.minimum(box1[3], boxes[:,3])

    # intersection : A & B
    intersection = np.maximum(0, x2-x1) * np.maximum(0, y2-y1)
    # union: A | B - A & B
    area1 = (box1[2]-box1[0])*(box1[3]-box1[1])
    area2 = (boxes[:,2]-boxes[:,0])*(boxes[:,3]-boxes[:,1])
    union = area1 + area2 - intersection
    # iou = intersection / union
    return intersection/union

In [ ]:
## Hungarian matching for Cost Optimization Problem
from scipy.optimize import linear_sum_assignment

def hungarian_matching(cost_matrix: np.ndarray, threshold: float) -> Tuple[int, int, int]:
    """Hungarian matching returns TP, FP, FN"""
    # 1. 行归约：每行减去最小值
    # 2. 列归约：每列减去最小值  
    # 3. 用最少的线覆盖所有的0
    # 4. 如果线数 < 矩阵维度，调整矩阵并重复
    # 5. 找到独立的0元素作为分配结果
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    # initialization of tp, fp, fn
    tp = fp = fn = 0
    matched_gt = set()
    matched_pred = set()

    for i,j in zip(row_ind, col_ind):
        if cost_matrix[i,j] <= threshold:
            tp += 1
            matched_gt.add(i)
            matched_pred.add(j)
    fp = len([j for j in range(cost_matrix.shape[1]) if j not in matched_pred])
    fn = len([i for i in range(cost_matrix.shape[0]) if i not in matched_gt])

    return tp, fp, fn

In [ ]:
from typing import Tuple
import numpy as np
def icp(source_points: np.ndarray, target_points: np.ndarray, max_iterations: int = 50, tolerance: float = 1e-6) -> Tuple[np.ndarray, np.ndarray]:
    transformation = np.eye(4)
    prev_error = 0
    for i in range(max_iterations):
        # 1. find correspondance per neareast neighbor
        correspondances = find_nearest_neighbors(source_points,target_points)
        # 2. optimial rigid transform
        R, t = compute_rigid_transform(source_points, target_points[correspondances])
        # 3. Apply trnasformation
        source_points = (R @ source_points.T).T + t
        # 4. update transfomration matrix
        current_transformation = np.eye(4)
        current_transformation[:3, :3] = R
        current_transformation[:3, 3] = t
        transformation = current_transformation @ transformation
        # 5. check convergence
        mean_error = np.mean(np.linalg.norm(source_points - target_points[correspondances], axis=1))
        if abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error
    return transformation, source_points, mean_error

def find_nearest_neighbors(source_points: np.ndarray, target_points: np.ndarray) -> np.ndarray:
    from scipy.spatial import cKDTree
    tree = cKDTree(target_points)
    distances, indices = tree.query(source_points, k=1)
    return indices

def compute_rigid_transform(A: np.ndarray, B: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    # offset to origin
    A_centered = A - centroid_A
    B_centered = B - centroid_B
    # calcualte convariance matrix
    H = A_centered.T @ B_centered
    # SVD decomposition
    U, S, Vt = np.linalg.svd(H)
    # compute rotation matrix
    R = Vt.T @ U.T
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T
    # compute translation
    t = centroid_B - R @ centroid_A
    return R, t


In [ ]:
def calculate_3d_iou(box1: Dict, box2: Dict) -> float:
    """    
    box:{
        'center': [x,y,z],
        'dimensions': [length, width, height], # l,w,h
        'rotation': [rx, ry, rz] # euler
    }
    """
    # 1. convert two 3d boxed to world coordinates
    corners1 = get_3d_box_corners(box1) 
    corners2 = get_3d_box_corners(box2) 

    # 2. compute intersection of 3d boxes
    intersection_volume = compute_3d_intersection(corners1, corners2)
    # 3. compute respective volume
    volume1 = box1['dimensions'][0] * box1['dimensions'][1] * box1['dimensions'][2]
    volume2 = box2['dimensions'][0] * box2['dimensions'][1] * box2['dimensions'][2]
    # 4. compute IoU
    union_volume = volume1 + volume2 - intersection_volume
    return intersection_volume / union_volume

def get_3d_box_corners(box: Dict, order: str = 'xyz') -> np.ndarray:
    """ obtain the eight corners of 3d boxes"""
    center = np.array(box['center'])
    l,w,h = box['dimensions']
    corners_local = np.array([
        [l/2, w/2, h/2], [l/2, w/2, -h/2],[l/2, -w/2, h/2], [l/2, -w/2, -h/2],
        [-l/2, w/2, h/2], [-l/2, w/2, -h/2],[-l/2, -w/2, h/2], [-l/2, -w/2, -h/2]
    ])
    rotation_matrix = euler_to_rotation_matrix(box['rotation'], str=order)
    corners_global = (rotation_matrix @ corners_local.T).T + center
    return corners_global

def euler_to_rotation_matrix(euler_angles: np.ndarray, order: str = 'zyx')
    """
    euler_angles: [roll, pitch, yaw] or [α, β, γ]
    order: e.g. 'zyx', 'xyz', etc
    """    
    roll, pitch, yaw = euler_angles
    Rx = np.array([
        [1,0,0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll), np.cos(roll)]
    ])
    Ry = np.array([
        [np.cos(pitch), 0, np.sin(pitch)],
        [0,1,0],
        [-np.sin(pitch), 0, np.cos(pitch)]
    ])
    Rz = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0,0,1]
    ])

    if order ==  'xyz': # 常见于机器人学 (偏航-俯仰-滚转)
        R = Rz @ Ry @ Rx    # 注意: 矩阵乘法从右到左应用
    elif order ==  'zyx':
        R = Rx @ Ry @ Rz
    else:
        raise ValueError(f'Unsupported rotation order: {order}')
    return R


In [ ]:
def farthest_point_sampling(points:np.ndarray, k:int) -> np.ndarray:
    # point to node partition with a fixed parition size
    n, dim  = points.shape
    # initialization
    centroids = np.zeros(k, dtype=int)
    distance = np.ones(n) * 1e10        # set inifinite to avoid false assignment
    farthest = np.random.randint(0,n)   # random farthest index initialization

    for i in range(k):
        centroids[i] = farthest
        centroid = points[farthest]
        # euclidians distance: points (N,D), centroid(D,), points - centroind (N,D)
        # np.sum(...,axis=1) -> sum along the feature dimension
        dist = np.sum((points - centroid) ** 2, axis=1)
        # update with the minimum distance
        mask = dist < distance
        distance[mask] = dist[mask]
        # chose the farthest point
        farthest = np.argmax(distance)

    return centroids

In [ ]:
# KD-Tree for KNN
from sklearn.neighbors import KDTree, NearestNeighbors
import numpy as np
from typing import List, Tuple

"""
    KNN算法原理：
    1. 计算查询点与所有点的距离
    2. 选择距离最小的k个点
    3. 返回这些点的索引和距离
"""

def knn(points:np.ndarray, query_point: np.ndarray, k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
    # initialize KD tree
    tree = KDTree(points, leaf_size=40)
    # query k nearest neighbor
    distances, indices = tree.query(query_point.reshape(1,-1), k=1)
    print(indices.shape, distances.shape)
    return indices[0], distances[0]

from scipy.spatial import KDTree as scipy_KDTree
def knn_scipy(points:np.ndarray, query_point: np.ndarray, k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
    tree = scipy_KDTree(points)
    distances, indices = tree.query(query_point, k=k)
    return indices, distances

# 使用示例
points = np.random.randn(1000, 3)  # 1000个3D点
query = np.array([0, 0, 0])
indices, distances = knn(points, query, k=5)
indices_scipy, distance_scipy = knn_scipy(points, query, k=5)
print(f"最近邻索引: {indices}")
print(f"距离: {distances}")
print(f"最近邻索引: {indices_scipy}")
print(f"距离: {distance_scipy}")

(1, 1) (1, 1)
最近邻索引: [388]
距离: [0.04164108]
最近邻索引: [388 269 916 141 412]
距离: [0.04164108 0.12194955 0.13840224 0.18172525 0.20388875]


In [ ]:
# conv2d

def conv2d(input_array: np.array, kernel: np.ndarray, stride: int = 1, padding: int = 0) -> np.ndarray:
    # check padding to preserve feature map and allow to include edges
    if padding > 0:
        # pad_width: ((before axis0, after axis0),(before axis1, after axis1))
        input_padded = np.pad(input_array,((padding, padding),(padding, padding)), mode='constant')
    else:
        input_padded = input_array
    
    H,W = input_padded.shape
    kH, kW = kernel.shape

    out_h = (H-kH) // stride + 1
    out_w = (W-kW) // stride + 1

    # initialzie output vector
    output = np.zeros((out_h, out_w))

    for i in range(out_h):
        for j in range(out_w):
            h_start = i * stride
            w_start = j * stride
            region = input_padded[h_start:h_start+kH, w_start:w_start+kW]
            output[i,j] = np.sum(region * kernel)
    return output


In [ ]:
# lane similarity camera

def lane_similarity_camera(lane1: np.ndarray, lane2: np.ndarray, image:np.ndarray = None) -> float:
    """lane1, lane2: (image_x,iamge_y)"""
    # geometric similarity
    geometric_sim = calculate_geometric_similarity(lane1, lane2) 
    # image appearance similatiry
    if image is not None:
        appearance_sim = calcualte_appearance_similarity(lane1, lane2, image)
        return 0.7 * geometric_sim + 0.3 * appearance_sim
    else:
        return geometric_sim

def calculate_geometric_similarity(lane1: np.ndarray, lane2: np.ndarray)-> float:
    # method 1: dynamic time windowing similairty for varied number of points
    if len(lane1) != len(lane2):
        return dtw_similairty(lane1, lane2)
    # method 2: inter point distance similarity
    point_distances = np.linalg.norm(lane1 - lane2, axis=1)
    avg_distance = np.mean(point_distances)
    # method 3: curvature similarity
    curvature_sim = curvature_similarity(lane1, lane2)
    direction_sim = direction_similarity(lane1, lane2)
    
    distance_sim = np.exp(-avg_distance / 10.0)
    geometric_sim = 0.5 * distance_sim + 0.3 * curvature_sim + 0.2 * direction_sim
    return geometric_sim

def dtw_similairty(lane1: np.ndarray, lane2: np.ndarray)-> float:
    from dtaidistance import dtw
    distance = dtw.distance(lane1, lane2)
    similarity = np.exp(-distance/max(len(lane1),len(lane2)))
    return similarity


In [ ]:
# lane similarity lidar

def lane_similarity_lidar(lane1_points: np.ndarray, lane2_points: np.ndarray, ground_plane:np.ndarray = None) -> float:
    """lane1_points, lane1_points: (N,3)"""
    # 1. spatial distance similarity
    spatial_sim = calcualte_spatial_similarity(lane1_points,lane2_points)
    # 2. structural similarity
    structural_sim = calculate_structural_similarity(lane1_points,lane2_points)
    # 3. ground projection similarity (if gorund plane polynomial is known)
    if ground_plane is not None:
        ground_sim = calculate_ground_projection_similarity(
            lane1_points, lane2_points, ground_plane
        )
        return 0.4 * spatial_sim + 0.4 * structural_sim + 0.2 * ground_sim
    else:
        return 0.6 * spatial_sim + 0.4 * structural_sim

def calcualte_spatial_similarity(points1: np.ndarray, points2: np.ndarray) -> float:
    # chamfer distance
    def chamfer_distance(p1,p2):
        from scipy.spatial import cKDTree
        tree1 = cKDTree(p1)
        tree2 = cKDTree(p2)
        dist1 = tree1.query(p2)[0]
        dist2 = tree2.query(p1)[0]
        return (np.mean(dist1)+np.mean(dist2))/2
    chamfer_dist = chamfer_distance(points1, points2)
    return np.exp(-chamfer_dist/0.5)

def calculate_structural_similarity(points1: np.ndarray, points2: np.ndarray) -> float:
    # 1. curvature similarity
    curvature1 = compute_curvature(points1)
    curvature2 = compute_curvature(points2)
    curvature_sim = 1.0 - abs(curvature1 - curvature2) / max(curvature1, curvature2)
    # direction similarity
    direction1 = compute_principal_direction(points1)
    direction2 = compute_principal_direction(points2)

    density_sim = calcualte_density_similarity(points1, points2)
    return 0.4 * curvature_sim + 0.4 * direction_sim + 0.2 * density_sim

from scipy import interpolate
def compute_curvature(points: np.ndarray) -> float:
    if (len(points) < 3):
        return 0.0
    # method 1: three points curvature
    curvatures = []
    for i in ranges(1, len(points)-1):
        p1, p2, p3 = points[i-1], points[i], points[i+1]
        curvature = curvature_from_three_points(p1, p2, p3)
        curvatures.append(curvature)
    return np.mean(curvatures) if curvatures else 0.0

def curvature_from_three_points(p1: np.ndarray, p2: np.ndarray, p3: np.ndarray) -> float:
    """通过三点计算曲率 (使用三角形外接圆方法)"""
    # compute triangle edges
    a = np.linalg.norm(p2-p3)
    b = np.linalg.norm(p1-p3)
    c = np.linalg.norm(p1-p2)

    # compute triagnle area
    s = (a+b+c)/2
    area = np.sqrt(s*(s-a)*(s-b)*(s-c))
    if area < 1e-10 or a*b*c < 1e-10:
        return 0.0
    # 曲率 = 4 * 面积 / (边长乘积)
    curvature = 4 * area / (a*b*c)

def curvature_similarity_simplified(lane1: np.ndarray, lane2: np.ndarray) -> float:
    """
    计算两条车道线的曲率相似度 - 使用滑动窗口
    返回: 0-1之间的相似度分数，1表示完全相同
    """
    curvature1 = compute_curvature(lane1)
    curvature2 = compute_curvature(lane2)

    if curvature1 == 0 and curvature2 == 0:
        return 1.0  # both straight
    # calculate relative difference
    max_curvature = max(abs(curvature1), abs(curvature2))
    curvature_diff = abs(curvature1 - curvature2)
    # similarity = 1 - normalized differences
    similarity = 1.0 - (curvature_diff/(max_curvature + 1e-8))
    return max(0.0, min(1.0, similarity))

def curvature_similarity(lane1: np.ndarray, lane2: np.ndarray, window_size: int = 5) -> float:
    """
    计算两条车道线的曲率相似度 - 使用滑动窗口
    返回: 0-1之间的相似度分数，1表示完全相同
    """
    if len(lane1) < window_size or len(lane2) < window_size:
        return curvature_similarity_simplified(lane1, lane2)
    
    curvatures1 = compute_curvature_sequence(lane1, window_size)
    curvatures2 = compute_curvature_sequence(lane2, window_size)

    # algin the length of curvature
    min_len = min(len(curvature1), len(curvature2))
    curvatures1 = curvatures1[:min_len]
    curvatures2 = curvatures2[:min_len]
    if min_len == 0:
        return 0.0
    correlation = np.corrcoef(curvatures1, curvatures2)[0,1]
    if np.isnan(correlation):
        return 0.0
    return (correlation + 1) / 2

def compute_curvature_sequence(points: np.ndarray, window_size: int = 5) -> np.ndarray:
    curvatures = []
    for i in range(len(points) - window_size + 1):
        window_points = points[i:i + window_size]
        curvature = compute_curvature(windows_points)
        curvatures.append(curvature)
    return np.array(curvatures)

def compute_direction(points: np.ndarray) -> float:
    if len(points) < 2:
        return 0.0
    start_point = points[0]
    end_point = points[-1]
    direction_vector = end_point - start_point
    # phi between y and x
    angle = np.arctan2(direction_vector[1], direction_vector[0])
    return angle

def compute_direction_sequence(points: np.ndarray, segment_length: int = 5) -> float:
    if(len(points) < segment_length):
        return np.array([compute_direction(points)])
    directions = []
    for i in range(0, len(points) - segment_length + 1, segment_length // 2):
        segment = points[i:i + segment_length]
        direction = compute_direction(segment)
        directions.append(direction)
    return np.array(directions)

def compute_principal_direction(points: np.ndarray) -> np.ndarray:
    """PCA"""
    if len(points) < 2:
        return np.array([1.0. 0.0])
    # 中心化点云
    centered = points - np.mean(points, axis=0)
    # 计算协方差矩阵
    cov_matrix = np.cov(centered.T)
    # 计算特征值和特征向量
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    # 选择最大特征值对应的特征向量 (主方向)
    principal_idx = np.argmax(eigenvalues)
    principal_direction = eigenvectors[:, principal_idx]
    # 确保方向一致性 (指向车道线延伸方向)
    if principal_direction[0] < 0: # 确保x分量为正
        principal_direction = -principal_direction
    return principal_direction

def direction_similarity(lane1: np.ndarray, lane2: np.ndarray) -> float:
    dir1 = compute_principal_direction(lane1)
    dir2 = compute_principal_direction(lane2)
    # compute angle cosine
    cosine_similarity = np.dot(dir1, dir2)
    return max(0.0, cosine_similarity)

def direction_similarity_detailed(lane1: np.ndarray, lane2: np.ndarray, segment_length: int = 5) -> float:
    dirs1 = compute_direction_sequence(lane1, segment_length)
    dirs2 = compute_direction_sequence(lane2, segment_length)

    min_len = min(len(dirs1), len(dirs2))
    if min_len == 0:
        return 0.0
    dirs1 = dirs1[:min_len]
    dirs2 = dirs2[:min_len]
    # 计算方向角度差异
    angle_diffs = np.abs(dirs1 - dirs2)
    # 处理角度周期性问题 (角度差应该在0-pi之间)
    angle_diffs = np.minimum(angle_diffs, 2*np.pi - angle_diffs)
    mean_angle_diff = np.mean(angle_diffs)
    # 将角度差异转换为相似度
    # 角度差为0时相似度为1，角度差为pi/2时相似度为0
    similarity = 1.0 - (mean_angle_diff / (np.pi/2))
    return max(0.0, min(1.0, similarity))

In [ ]:
from typing import List, Tuple, Dict
import numpy as np

def match_lanes_hugarian(
        gt_lanes: List[np.ndarray],
        pred_lanes: List[np.ndarray],
        threshold: float = 0.8
) -> Dict[str, float]:
    # initialize cost matrix
    cost_matrix = np.zeros(len(gt_lanes), len(pred_lanes))
    for i, gt_lane in enumerate(gt_lanes):
        for j, pred_lane in enumerate(pred_lanes):
            cost_matrix[i,j] = 1- lane_similarity(gt_lane, pred_lane)
    # hugarian matching
    from scipy.optimize import linear_sum_assignment
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    # tp, fp, fn
    tp = fp = fn = 0
    for i,j in zip(row_ind, col_ind):
        if cost_matrix[i,j] <= (1-threshold):
            tp += 1
        else:
            fn += 1
            fp += 1
    # precision, recall, f1
    precision = tp /(tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

In [ ]:
def fit_lane_polynomial(points: np.ndarray, degree: int = 3) -> np.ndarray:
    x = points[:,0]
    y = points[:,1]
    coefficients = np.polyfit(y, x, degree)
    return coefficients

def calculate_lane_curvature(coefficients: np.ndarray, y:float) -> float:
    # curvature formula: k = |f''(y)| / (1 + f'(y)^2)^(3/2)
    poly = np.poly1d(coefficients)
    first_deriv = poly.deriv()(y)
    second_deriv = poly.deriv(2)(y)
    curvature = np.abs(second_deriv) / (1 + first_deriv ** 2) ** 1.5

In [ ]:
import numpy as np
def ransac_ground_segmentation(points:np.ndarray, num_iteration: int = 100, distance_threshold: float = 0.1) -> Tuple[np.ndarray, np.ndarray]:
    best_plane = None
    best_inliers = []

    for _ in range(num_iteration):
        # three points formulates a plane
        sample_indices = np.random.choice(len(points), 3, replace=False)
        sample_points = points[sample_indices]
        # formualte plane Ax + By + Cz + D = 0
        v1 = sample_points[1] - sample_points[0]
        v2 = sample_points[2] - sample_points[0]
        normal = np.cross(v1, v2)
        normal = normal / (np.linalg.norm(normal)+ 1e-8)
        D = -np.dot(normal, sample_points[0])
        # point-to-plane distance
        distances = np.abs(points @ normal + D) / np.linalg.norm(normal)
        # find inliers
        inliers = np.where(distances < distance_threshold)[0]
        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_plane = (normal, D)
    ground_points = points[best_inliers]
    non_ground_points = points[~np.isin(np.arange(len(points)), best_inliers)]
    return ground_points, non_ground_points

In [ ]:
import cv2
# lidar to camer projection
def lidar_to_camera_projection(lidar_points: np.ndarray, camera_matrix: np.ndarray, dist_coeffs: np.ndarray, extrinsic_matrix: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    激光雷达坐标系 
    ↓ 外参矩阵
    相机坐标系  
    ↓ 去除畸变
    归一化相机坐标系
    ↓ 内参矩阵  
    图像坐标系 (像素坐标)
    内参矩阵：描述相机坐标系到图像坐标系的变换
    外参矩阵：描述世界坐标系到相机坐标系的变换
            坐标变换：世界点 -> 相机点
            坐标变换公式：
            # point_camera = R @ point_world + t
            但代码是：point_camera = point_lidar @ R.T + t
            如果 extrinsic_matrix 表示从相机到激光雷达的变换 (camera_to_lidar)
            那么从激光雷达到相机需要取逆：
            # R_lidar_to_camera = R_camera_to_lidar.T
            # t_lidar_to_camera = -R_camera_to_lidar.T @ t_camera_to_lidar
    畸变系数：描述真实相机镜头与理想针孔模型的偏差
    """
    # 1. coordinate system transformation: lidar coord system to camera coord system
    points_camera = lidar_points @ extrinsic_matrix[:3, :3].T + extrinsic_matrix[:3, 3]
    # 2. remove points beyond camera screen
    mask = points_camera[:, 2] > 0
    points_camera = points[mask]
    # project to 2d plane
    points_2d, _ = cv2.projectPoints(points_camera, np.zeros(3), np.zeros(3), dist_coeffs)
    points_2d = points_2d.reshape(-1,2) # 从(N,1,2)变为(N,2)
    # filter points outside of image boarder
    height, width = 1080, 1920
    valid_mask = (points_2d[:,0] >= 0) & (points_2d[:,0] <= width) & (points_2d[:,1] >= 0) & (points_2d[:,1] <= height)
    return points_2d[valid_mask], valid_mask

In [ ]:
# Self Attention
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int, num_heads: int = 8):
        # instantiate module interface
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads 
        assert self.head_dim * num_heads == embed_dim
        # instantiate linear transform layers (N,N)
        self.qkv = nn.Linear(embed_dim, embed_dim * 3)
        self.proj = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x:torch.Tensor) -> torch.Tensor:
        # decode input dimensions
        batch_size, seq_len, embed_dim = x.shape
        # encode query, key, and value
        qkv = self.qkv(x)                                                           # [batch_size, seq_len, 3 * embed_dim]
        # instantiate multihead 
        qkv = qkv.reshape(batch_size, seq_len, 3, self.num_heads, self.head_dim)    # [batch_size, seq_len, 3, num_heads, head_dim]
        qkv = qkv.permute(2, 0, 3, 1, 4)                                            # [3, batch, num_heads, seq_len, head_dim]
        q, k, v = qkv[0], qkv[1], qkv[2]                                            # 每个都是 [batch_size, num_heads, seq_len, head_dim]
        # compute attention weights
        scale = self.head_dim ** -0.5
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) * scale                  # [batch_size, num_heads, seq_len, seq_len]
        attn_weight = F.softmax(attn_scores, dim=-1)                                # [batch_size, num_heads, seq_len, seq_len]
        # apply the attention weights
        output = torch.matmul(attn_weight, v)                                       # [batch_size, num_heads, seq_len, head_dim]
        output = output.transpose(1,2).contiguous.view(batch_size, seq_len, embed_dim) # [batch_size, seq_len, num_heads * head_dim]
        output = self.proj(output)
        return output


In [ ]:
from typing import Optional, List
from collections import deque

def level_order_traversal(root: Optional['TreeNode']) -> List[List[int]]:
    """二叉樹層次遍歷"""
    if not root:
        return []
    
    result = []
    queue = deque([root])
    
    while queue:
        level_size = len(queue)
        level_nodes = []
        
        for _ in range(level_size):
            node = queue.popleft()
            level_nodes.append(node.val)
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)
        
        result.append(level_nodes)
    
    return result

def reverse_linked_list(head: Optional['ListNode']) -> Optional['ListNode']:
    """鏈表反轉"""
    prev = None
    current = head
    
    while current:
        next_temp = current.next
        current.next = prev
        prev = current
        current = next_temp
    
    return prev

In [ ]:
def min_path_sum(grid: List[List[int]]) -> int:
    """最小路徑和"""
    if not grid or not grid[0]:
        return 0
    
    m, n = len(grid), len(grid[0])
    dp = [[0] * n for _ in range(m)]
    dp[0][0] = grid[0][0]
    
    for i in range(1, m):
        dp[i][0] = dp[i-1][0] + grid[i][0]
    for j in range(1, n):
        dp[0][j] = dp[0][j-1] + grid[0][j]
    
    for i in range(1, m):
        for j in range(1, n):
            dp[i][j] = min(dp[i-1][j], dp[i][j-1]) + grid[i][j]
    
    return dp[m-1][n-1]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PointNetFeatureExtractor(nn.Module):
    """PointNet特征提取器 - 3D深度学习的基石"""
    def __init__(self, feature_dim: int = 1024):
        super().__init__()
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, feature_dim, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(feature_dim)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """x: (B, 3, N)"""
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]  # Max pooling
        x = x.view(-1, 1024)
        return x

class Transformer3D(nn.Module):
    """3D点云Transformer"""
    def __init__(self, input_dim: int = 3, hidden_dim: int = 256, num_heads: int = 8):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, hidden_dim)
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads)
        self.norm1 = nn.LayerNorm(hidden_dim)
        self.norm2 = nn.LayerNorm(hidden_dim)
        self.ffn = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 4),
            nn.ReLU(),
            nn.Linear(hidden_dim * 4, hidden_dim)
        )
        
    def forward(self, points: torch.Tensor) -> torch.Tensor:
        """points: (N, 3)"""
        x = self.input_proj(points)  # (N, hidden_dim)
        
        # Self-attention
        attn_out, _ = self.attention(x, x, x)
        x = self.norm1(x + attn_out)
        
        # Feed-forward
        ff_out = self.ffn(x)
        x = self.norm2(x + ff_out)
        
        return x

In [ ]:
def temporal_synchronization(lidar_data: Dict, camera_data: Dict, 
                           gps_data: Dict, timestamp_tolerance: float = 0.01) -> Dict:
    """
    多传感器时间同步 - 实际工程中的关键问题
    """
    # 1. 找到时间戳最接近的数据帧
    lidar_ts = lidar_data['timestamp']
    camera_ts = camera_data['timestamp']
    gps_ts = gps_data['timestamp']
    
    # 2. 检查时间同步性
    max_diff = max(abs(lidar_ts - camera_ts), abs(lidar_ts - gps_ts))
    if max_diff > timestamp_tolerance:
        print(f"警告：传感器时间不同步，最大差异: {max_diff:.3f}s")
    
    # 3. 应用时间补偿（如果知道传感器延迟）
    compensated_data = apply_time_compensation(
        lidar_data, camera_data, gps_data
    )
    
    return compensated_data

def spatial_calibration(lidar_points: np.ndarray, camera_matrix: np.ndarray,
                       lidar_to_camera_transform: np.ndarray) -> np.ndarray:
    """
    激光雷达到相机的空间标定
    """
    # 1. 坐标变换: 激光雷达坐标系 -> 相机坐标系
    points_camera = (lidar_to_camera_transform[:3, :3] @ lidar_points.T + 
                    lidar_to_camera_transform[:3, 3].reshape(3, 1)).T
    
    # 2. 投影到图像平面
    points_2d = (camera_matrix @ points_camera.T).T
    points_2d = points_2d[:, :2] / points_2d[:, 2:3]  # 齐次坐标归一化
    
    return points_2d

In [ ]:
import torch
import torch.nn as nn

class LaneSimilarityNet(nn.Module):
    """深度学习车道线相似度网络"""
    
    def __init__(self, feature_dim: int = 128):
        super().__init__()
        self.lane_encoder = nn.Sequential(
            nn.Linear(6, 64),  # 假设每个点有(x,y,z,curvature,direction,confidence)
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(), 
            nn.Linear(128, feature_dim)
        )
        
        self.similarity_head = nn.Sequential(
            nn.Linear(feature_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, lane1: torch.Tensor, lane2: torch.Tensor) -> torch.Tensor:
        # 编码车道线特征
        feat1 = self.lane_encoder(lane1).mean(dim=0)  # 全局平均池化
        feat2 = self.lane_encoder(lane2).mean(dim=0)
        
        # 计算相似度
        combined = torch.cat([feat1, feat2], dim=-1)
        similarity = self.similarity_head(combined)
        
        return similarity

🕒 長時間學習計劃（12-14小時）

第一階段：核心突破 (4小時)

08:00-09:30：NMS + IoU + 匈牙利匹配 (反覆練習3遍)
09:30-10:30：點雲地面分割 + FPS
10:30-12:00：車道線檢測評估指標
第二階段：重點強化 (4小時)

13:00-14:30：KNN + KD-Tree + Conv2D
14:30-16:00：C++底層八股複習
16:00-17:00：項目技術細節準備
17:00-18:00：坐標變換 + 多傳感器融合
第三階段：基礎鞏固 (3小時)

19:00-20:00：二叉樹 + 鏈表 + 動態規劃
20:00-21:00：所有題目第二遍練習
21:00-22:00：模擬面試 + 弱項強化
第四階段：衝刺複習 (2小時)

22:00-23:00：重點題目第三遍練習
23:00-24:00：面經問題自問自答
💡 學習建議

每題都要手寫，不要只看不寫
先理解思路，再寫代碼
注重邊界條件和錯誤處理
練習口頭解釋算法思路
定期休息，保持精力